# Plantilla: Proyecto
## Tema 2: Clasificación de barrios en función de su tipología.
Documento plantilla para el proyecto con los imports de las librerias más importantes.

In [1]:
import nltk                      # Natural Language Toolkit
import scrapy                    # Web scraping   
import requests                  # HTTP requests
import selenium                  # Web automation
import numpy as np               # Numerical computing
import pandas as pd              # Data manipulation
import seaborn as sns            # Data visualization
import tensorflow as tf          # Machine learning
import matplotlib.pyplot as plt  # Data visualization
import os                        # Operating system

!pip install openpyxl
import openpyxl                  # Excel files

# Automatización en la recogida de variables (dataset Barris)

In [2]:
# tengo un .csv en el path ./data/barrios/barrios_codigos.csv. Me interesan las columnas Nombre, Codigo distrito y Codigo barrio, quédate con esas.

idBarrios = pd.read_csv('./data/barrios/barrios_codigos.csv', delimiter=';')
idBarrios = idBarrios[['Nombre', 'Codigo distrito', 'Codigo barrio']]

barrios_dict = {}

for index, row in idBarrios.iterrows():
    nombre = row['Nombre']
    codigo_distrito = row['Codigo distrito']
    codigo_barrio = row['Codigo barrio']
    
    barrio_dict = {
        'Codigo distrito': codigo_distrito,
        'Codigo barrio': codigo_barrio
    }
    
    barrios_dict[nombre] = barrio_dict

In [3]:
def get_barrio_name(codigo_distrito, codigo_barrio):
    for nombre, barrio_dict in barrios_dict.items():
        if barrio_dict['Codigo distrito'] == codigo_distrito and barrio_dict['Codigo barrio'] == codigo_barrio:
            return nombre

In [4]:
sheets = pd.ExcelFile('./data/barrios/2024/Districte_01_Barri_1.xlsx').sheet_names
print(sheets)

['Padrón', 'MovPadrón', 'Censo', 'Vehículos', 'Catastro', 'ActEconom', 'Elecciones']


In [5]:
def find_string_in_df(df, string):
    for i in range(len(df)):
        if string in str(df.iloc[i, 0]):
            return i
    return None

def find_skip(df, row):
    # Encontrar la primera fila que no esté completamente llena de NAs después de la fila especificada
    for i in range(row + 2, len(df)): # el +2 en lugar de +1 es para poder ponerlo en castellano xD
        if not df.iloc[i].isna().all():
            return i - row

    return 0  # Si todas las filas después de 'row' están completamente llenas de NAs, skip es 0

def extract_df(df, row, skip, n=None):
    if n is None:
        # Encontrar n dinámicamente.
        last_row = row + skip
        while last_row < len(df) and not df.iloc[last_row].isna().all():
            last_row += 1
        n = last_row - row - skip
    extracted_df = df.iloc[row + skip:row + skip + n, :]
    
    # Verificar si TODAS las filas en cada columna son NA
    columns_to_drop = extracted_df.columns[extracted_df.isna().all()]
    
    # Eliminar solo las columnas donde todas las filas son NA
    extracted_df = extracted_df.drop(columns_to_drop, axis=1)
    
    return extracted_df

def extract_df_by_string(df, string, n=None):
    row = find_string_in_df(df, string)
    skip = find_skip(df, row)
    return extract_df(df, row, skip, n)

In [6]:
barrioPadron = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'Padrón')
barrioMovPadron = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'MovPadrón')
barrioVehiculos = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'Vehículos')
barrioCatastro = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'Catastro')
barrioElecciones = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'Elecciones')

In [82]:
def extract_dataframes_from_excel(excel_path):
    barrioPadron = pd.read_excel(excel_path, sheet_name = 'Padrón')
    barrioMovPadron = pd.read_excel(excel_path, sheet_name = 'MovPadrón')
    barrioVehiculos = pd.read_excel(excel_path, sheet_name = 'Vehículos')
    barrioCatastro = pd.read_excel(excel_path, sheet_name = 'Catastro')
    barrioElecciones = pd.read_excel(excel_path, sheet_name = 'Elecciones')

    dfSup = extract_df_by_string(barrioPadron, string='Superficie y densidad de población')
    dfSup.columns = dfSup.iloc[0]
    dfSup = dfSup.iloc[1:]
    dfSup.reset_index(drop=True, inplace=True)
    
    
    dfPobSexEdad = extract_df_by_string(barrioPadron, string='Población por sexo y edad (grandes grupos)')
    dfPobSexEdad.columns = dfPobSexEdad.iloc[0]
    dfPobSexEdad = dfPobSexEdad.iloc[1:]
    dfPobSexEdad.columns = ['Género'] + list(dfPobSexEdad.columns[1:])
    dfPobSexEdad.reset_index(drop=True, inplace=True)
    
    
    dfPobSexLugar = extract_df_by_string(barrioPadron, string='Población según lugar de nacimiento y sexo')
    dfPobSexLugar.columns = dfPobSexLugar.iloc[0]
    dfPobSexLugar = dfPobSexLugar.iloc[1:]
    dfPobSexLugar.columns = ['Género'] + list(dfPobSexLugar.columns[1:])
    dfPobSexLugar.reset_index(drop=True, inplace=True)
    
    dfPobExtCont = extract_df_by_string(barrioPadron, string='Población extranjera según nacionalidad, por contienentes')
    dfPobExtCont.columns = dfPobExtCont.iloc[0]
    dfPobExtCont = dfPobExtCont.iloc[1:]
    dfPobExtCont.columns = ['Género'] + list(dfPobExtCont.columns[1:])
    dfPobExtCont.reset_index(drop=True, inplace=True)

    dfHojasFamComp = extract_df_by_string(barrioPadron, string='Número de hojas familiares según composición')
    dfHojasFamComp.columns = dfHojasFamComp.iloc[0]
    dfHojasFamComp = dfHojasFamComp.iloc[1:]
    dfHojasFamComp.reset_index(drop=True, inplace=True)
    
    dfIndicadoresDemograficosPadron = extract_df_by_string(barrioPadron, string='Indicadores demográficos')
    dfIndicadoresDemograficosPadron.columns = dfIndicadoresDemograficosPadron.iloc[0]
    dfIndicadoresDemograficosPadron = dfIndicadoresDemograficosPadron.iloc[1:]
    dfIndicadoresDemograficosPadron.reset_index(drop=True, inplace=True)

    dfMovPadron = extract_df_by_string(barrioMovPadron, string='Resumen de movimientos registrados en el Padrón Municipal')
    dfMovPadron.columns = dfMovPadron.iloc[0]
    dfMovPadron = dfMovPadron.iloc[1:]
    dfMovPadron.reset_index(drop=True, inplace=True)
    
    dfIndicadoresDemograficosMovPadron = extract_df_by_string(barrioMovPadron, string='Indicadores demográficos')
    dfIndicadoresDemograficosMovPadron.columns = dfIndicadoresDemograficosMovPadron.iloc[0]
    dfIndicadoresDemograficosMovPadron = dfIndicadoresDemograficosMovPadron.iloc[1:]
    dfIndicadoresDemograficosMovPadron.reset_index(drop=True, inplace=True)
    
    
    dfVehicTipo = extract_df_by_string(barrioVehiculos, string='Vehículos según tipo')
    dfVehicTipo.columns = dfVehicTipo.iloc[0]
    dfVehicTipo = dfVehicTipo.iloc[1:]
    dfVehicTipo.columns = ['Tipo de vehículo'] + list(dfVehicTipo.columns[1:])
    dfVehicTipo.reset_index(drop=True, inplace=True)
    
    dfTurismPot = extract_df_by_string(barrioVehiculos, string='Turismos según potencia')
    dfTurismPot.columns = dfTurismPot.iloc[0]
    dfTurismPot = dfTurismPot.iloc[1:]
    dfTurismPot.columns = ['Potencia'] + list(dfTurismPot.columns[1:])
    dfTurismPot.reset_index(drop=True, inplace=True)

    dfBienesInmAno = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles según el año de antigüedad')
    dfBienesInmAno.columns = dfBienesInmAno.iloc[0]
    dfBienesInmAno = dfBienesInmAno.iloc[1:]
    dfBienesInmAno.reset_index(drop=True, inplace=True)
    
    dfBienesInmSup = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles construidas después de 1800 según superficie construida')
    dfBienesInmSup.columns = dfBienesInmSup.iloc[0]
    dfBienesInmSup = dfBienesInmSup.iloc[1:]
    dfBienesInmSup.reset_index(drop=True, inplace=True)
    
    dfBienesInmVal = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles construidas después de 1800 según valor catastral')
    dfBienesInmVal.columns = dfBienesInmVal.iloc[0]
    dfBienesInmVal = dfBienesInmVal.iloc[1:]
    dfBienesInmVal.reset_index(drop=True, inplace=True)
    
    
    dfBienesValMedio = extract_df_by_string(barrioCatastro, string='Valores catastrales medios')
    dfBienesValMedio.columns = dfBienesValMedio.iloc[0]
    dfBienesValMedio = dfBienesValMedio.iloc[1:]
    dfBienesValMedio.reset_index(drop=True, inplace=True)
    

    dfSupAparc = extract_df_by_string(barrioCatastro, string='Superficie total de los aparcamientos')
    dfSupAparc.columns = dfSupAparc.iloc[0]
    dfSupAparc = dfSupAparc.iloc[1:]
    dfSupAparc.reset_index(drop=True, inplace=True)
    
    dfElecGen = extract_df_by_string(barrioElecciones, string='Votos a candidaturas')
    dfElecGen.columns = dfElecGen.iloc[0]
    dfElecGen = dfElecGen.iloc[1:]
    dfElecGen.columns = ['Partido'] + list(dfElecGen.columns[1:])
    dfElecGen.reset_index(drop=True, inplace=True)
    
    
    dataframes = { 'dfSup': dfSup,
                     'dfPobSexEdad': dfPobSexEdad,
                     'dfPobSexLugar': dfPobSexLugar,
                     'dfPobExtCont': dfPobExtCont,
                     'dfHojasFamComp': dfHojasFamComp,
                     'dfIndicadoresDemograficosPadron': dfIndicadoresDemograficosPadron,
                     'dfMovPadron': dfMovPadron,
                     'dfIndicadoresDemograficosMovPadron': dfIndicadoresDemograficosMovPadron,
                     'dfVehicTipo': dfVehicTipo,
                     'dfTurismPot': dfTurismPot,
                     'dfBienesInmAno': dfBienesInmAno,
                     'dfBienesInmSup': dfBienesInmSup,
                     'dfBienesInmVal': dfBienesInmVal,
                     'dfBienesValMedio': dfBienesValMedio,
                     'dfSupAparc': dfSupAparc,
                     'dfElecGen': dfElecGen
                  }
    
    return dataframes

In [18]:
def clean_dataframe(df):
    # Eliminar filas y columnas con todos los valores NaN
    df = df.dropna(axis=0, how='all').dropna(axis=1, how='all')
    # Establecer la primera fila como nombres de columnas y reindexar
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index(drop=True)
    return df

def modify_first_column_name(df, column_name):
    df.columns = [column_name] + list(df.columns[1:])
    return df
        
def extract_dataframes_from_excel(excel_path):
    dataframes = {}
    sheets = {
        'Padrón': ['Superficie y densidad de población', 'Población por sexo y edad (grandes grupos)',
                   'Población según lugar de nacimiento y sexo', 'Población extranjera según nacionalidad, por contienentes',
                   'Número de hojas familiares según composición', 'Indicadores demográficos'],
        'MovPadrón': ['Resumen de movimientos registrados en el Padrón Municipal', 'Indicadores demográficos'],
        'Vehículos': ['Vehículos según tipo', 'Turismos según potencia'],
        'Catastro': ['Bienes Inmuebles según el año de antigüedad',
                     'Bienes Inmuebles construidas después de 1800 según superficie construida',
                     'Bienes Inmuebles construidas después de 1800 según valor catastral',
                     'Valores catastrales medios', 'Superficie total de los aparcamientos'],
        'Elecciones': ['Votos a candidaturas']
    }
    
    for sheet, strings in sheets.items():
        df_sheet = pd.read_excel(excel_path, sheet_name=sheet)
        for string in strings:
            df = extract_df_by_string(df_sheet, string)
            if df is not None:
                df = clean_dataframe(df)
                if sheet == 'Padrón' and string in ['Población por sexo y edad (grandes grupos)', 'Población según lugar de nacimiento y sexo', 'Población extranjera según nacionalidad, por contienentes']:
                    modify_first_column_name(df, 'Género')
                elif sheet == 'Vehículos' and string == 'Vehículos según tipo':
                    modify_first_column_name(df, 'Tipo de vehículo')
                elif sheet == 'Elecciones' and string == 'Votos a candidaturas':
                    modify_first_column_name(df, 'Partido')
                dataframes[f'{sheet}_{string.replace(" ", "_")}'] = df
    
    return dataframes


In [19]:
def get_barrio_name_from_filename(filename, barrios_dict):
    # Extraer el nombre del distrito y el número de barrio del nombre del archivo
    distrito_numero, barrio_numero = os.path.splitext(filename)[0].split('_')[1:4][::2]
    
    # Buscar el nombre del distrito y el número de barrio en barrios_dict
    for nombre, info_barrio in barrios_dict.items():
        if info_barrio['Codigo distrito'] == int(distrito_numero) and info_barrio['Codigo barrio'] == int(barrio_numero):
            return nombre
    
    return None

In [20]:
# Ahora quiero que vayas a la carpeta ./data/barrios/2024 y leas todos los archivos .xlsx que haya en ella. Para cada archivo, quiero que extraigas los dataframes que te interesan y los guardes en un diccionario. La clave del diccionario será el nombre del barrio (extraído del nombre del archivo) y el valor será otro diccionario con los dataframes extraídos.

datos_2024 = {}

for filename in os.listdir('./data/barrios/2024'):
    if filename.endswith('.xlsx'):
        excel_path = os.path.join('./data/barrios/2024', filename)
        dataframes = extract_dataframes_from_excel(excel_path)
        barrio_name = get_barrio_name_from_filename(filename, barrios_dict)
        datos_2024[barrio_name] = dataframes

In [26]:
datos_2024['ALBORS']['Padrón_Población_extranjera_según_nacionalidad,_por_contienentes'].head()

,Género,Total,Resta UE(27),Europa No UE(27),Àfrica,Amèrica del Nord,Amèrica Central,Amèrica del Sud,Àsia
0,Total,1565,489,111,94,27,96,498,250
1,Homes,736,253,34,53,15,26,217,138
2,Dones,829,236,77,41,12,70,281,112
